<font color='orange' size=8><B>Lab Report : Basic ROS for FRA333</B></font>

<font size=5><B>Objective:</B></font>
- เพื่อศึกษาการทำงานของระบบ ROS2 ในหัวข้อ Node, Publisher, Subscriber, Service, Parameter และ Launch File
- เพื่อให้นักศึกษาสามารถประยุกต์การองค์ความรู้จากวัตถุประสงค์ข้อที่ 1 กับโจทย์พื้นฐาน

------------------------------------------------------------------------------------------------------------------------------

<img src="pic1.png" />

- <font color='red'>Node: /linear/noise_generator</font> คือ โหนดที่สามารถสร้างสัญญาณรบกวนของความเร็วเชิงเส้นจำนวน 1 ช่องสัญญาณประกอบด้วย 
    - <font color='green'>Topic (Publisher): /noise Type: std_msgs/Float64</font> 
    - <font color='pink'>Argument System: /rate</font> ความถี่ในการส่งสัญญาณ นักศึกษาสามารถตั้งค่าความถี่ได้จาก launch file (The default rate is 5.0 Hz)
    - <font color='blue'>Service (Server): /set_noise</font> สำหรับการปรับค่าคุณสมบัติของสัญญาณรบกวน โดยที่การเรียกใช้งาน service ผ่าน custom service ในรูปแบบดังนี้ (ไฟล์ SetNoise.srv ใน lab1_interfaces/srv)
        - <font color='purple'>request: /mean (std_msgs/Float64)</font> (The default mean is 0.0)
        - <font color='purple'>request: /variance (std_msgs/Float64)</font> (The default mean is 1.0)

- <font color='red'>Node: /angular/noise_generator</font> คือ โหนดที่สามารถสร้างสัญญาณรบกวนของความเร็วเชิงมุมจำนวน 1 ช่องสัญญาณประกอบด้วย 
    - <font color='green'>Topic (Publisher): /noise Type: std_msgs/Float64</font> 
    - <font color='pink'>Argument System: /rate</font> ความถี่ในการส่งสัญญาณ นักศึกษาสามารถตั้งค่าความถี่ได้จาก launch file (The default rate is 5.0 Hz)
    - <font color='blue'>Service (Server): /set_noise</font> สำหรับการปรับค่าคุณสมบัติของสัญญาณรบกวน โดยที่การเรียกใช้งาน service ผ่าน custom service ในรูปแบบดังนี้ (ไฟล์ SetNoise.srv ใน lab1_interfaces/srv)
        - <font color='purple'>request: /mean (std_msgs/Float64)</font> (The default mean is 0.0)
        - <font color='purple'>request: /variance (std_msgs/Float64)</font> (The default mean is 1.0)

- <font color='red'>Node: /velocity_multiplexer</font> คือ โหนดที่สามารถรวมสัญญาณรบกวนจากโหนด <font color='red'>Node: /linear/noise_generator และ /angular/noise_generator</font> จากนั้นนำมาสร้างคำสั่งในการเคลื่อนที่ของเต่า
    - <font color='green'>Topic (Publisher): /cmd_vel Type: geometry_msgs/Twist</font> 
    - <font color='green'>Topic (Subscriber): /linear/noise Type: std_msgs/Float64</font> 
    - <font color='green'>Topic (Subscriber): /angular/noise Type: std_msgs/Float64</font> 
    - <font color='pink'>Argument System: /rate</font> ความถี่ในการส่งสัญญาณ นักศึกษาสามารถตั้งค่าความถี่ได้จาก launch file (The default rate is 5.0 Hz)

------------------------------------------------------------------------------------------------------------------------------

<font color='orange' size=6><B>อธิบายการเขียน Node: noise_generator และ velocity_mux</B></font>

- <font color='red'>Node: /linear,angular/noise_generator</font> คือ โหนดที่สามารถสร้างสัญญาณรบกวนของความเร็วเชิงเส้นจำนวน 1 ช่องสัญญาณประกอบด้วย 
    - <font color='green'>Topic (Publisher): /noise Type: std_msgs/Float64</font> 
    - <font color='pink'>Argument System: /rate</font> ความถี่ในการส่งสัญญาณ นักศึกษาสามารถตั้งค่าความถี่ได้จาก launch file (The default rate is 5.0 Hz)
    - <font color='blue'>Service (Server): /set_noise</font> สำหรับการปรับค่าคุณสมบัติของสัญญาณรบกวน โดยที่การเรียกใช้งาน service ผ่าน custom service ในรูปแบบดังนี้ (ไฟล์ SetNoise.srv ใน lab1_interfaces/srv)
        - <font color='purple'>request: /mean (std_msgs/Float64)</font> (The default mean is 0.0)
        - <font color='purple'>request: /variance (std_msgs/Float64)</font> (The default mean is 1.0)

- <font color='red'>การ Import library ที่จำเป็นในการเขียน Node</font>
    - <font color='green'>numpy </font>สำหรับการใช้ Function Random (np.random.normal())
    - <font color='green'>rclpy.node import Node </font>เพื่อให้สามารถสร้าง Class Node inherits function ต่าง ๆ มาได้
    - <font color='green'>std_msgs.msg </font> เพื่อให้สามารถใช้ msg type Float64 ตาม requirement ได้
    - <font color='green'>lab1_interfaces.srv import SetNoise </font> เพื่อให้สามารถใช้ custom srv จาก pkg lab1_interfaces ได้ (SetNoise)

In [1]:
import rclpy
import numpy as np
from rclpy.node import Node
from std_msgs.msg import Float64

from lab1_interfaces.srv import SetNoise

import sys

- <font color='red'>Class NoiseGenerator</font>

In [2]:
class NoiseGenerator(Node):

    def __init__(self):
        super().__init__('noise_generator')
        # get the rate from argument or default
        if len(sys.argv)>2: 
            self.rate = float(sys.argv[1])
        else:
            self.rate = 5.0
        # add codes here
        
        # Parameters
        self.declare_parameters(namespace='',parameters=[('None',None)]) 
        self.noise_type = self.get_namespace()
        self.noise_rate = 1/self.rate # rate is the publishing rate defualt is 5 HZ (0.2s)
        
        # Publishers
        self.noise_pub = self.create_publisher(Float64, f'{self.noise_type}/noise', 10)
        
        # Services Server
        self.noise_server = self.create_service(SetNoise, f'{self.noise_type}/set_noise', self.set_noise_callback)
        
        # Timer
        self.timer = self.create_timer(self.noise_rate, callback=self.timer_callback)

        # additional attributes
        # variables
        self.noise_data = 0.0
        self.mean = 0.0
        self.variance = 1.0
        self.get_logger().info(f'Starting {self.get_namespace()}/{self.get_name()} with the default parameter. mean: {self.mean}, variance: {self.variance}')
    
    def set_noise_callback(self,request:SetNoise.Request,response:SetNoise.Response):
        # Set the noise parameter
        # Service callback with srv request and response
        # SetNoise.srv
        # request = float64 mean , float64 variance
        # response = None
        self.mean = request.mean.data
        self.variance = request.variance.data
        return response # return response to the client
    
    def publish_noise_msg(self):
        # Generate noise and publish
        # Foat64 msg : float64 data
        self.noise_data = np.random.normal(self.mean, np.sqrt(self.variance))  # Gaussian noise
        noise = Float64()
        noise.data = self.noise_data
        self.noise_pub.publish(noise)
    
    def timer_callback(self):
        self.publish_noise_msg()
        self.get_logger().info('node %s noise data: %.4f' % (self.noise_type,self.noise_data))

1.โค้ตที่ใช้ในการ initialize class node โดยจะตั้งชื่อให้กับ node ของ class นี้ว่า noise_generator

    def __init__(self):
        super().__init__('noise_generator')

2.กำหนดให้ node รับค่า argument โดยจะทำการเช็คว่าได้รับรับมาหรือไม่หากได้รับมาจะทำการ set ค่าของ rate หากไม่ได้รับจะตั้งเป็นค่า Defult(5.0 hz)\
(sys.argv[0] คือ path)

        # get the rate from argument or default
        if len(sys.argv)>2: 
            self.rate = float(sys.argv[1])
        else:
            self.rate = 5.0
        # add codes here

3.ใช้ function declare_parameters ในการประกาศให้ node มี parameters และ namespace เริ่มต้น 'None'เป็น None และ namespace เป็น ''(ไม่มี) จากนั้น
ทำการดึงค่า namespace โดยใช้ function get_namespace() มาเก็บไว้ที่ตัวแปร noise_type และทำการแปลง rate จากความถี่ให้กลายเป็นคาบ

        self.declare_parameters(namespace='',parameters=[('None',None)]) # ไม่จำเป็นต้องประกาศ
        self.noise_type = self.get_namespace()
        self.noise_rate = 1/self.rate # rate is the publishing rate defualt is 5 HZ (0.2s)

4.ประกาศ publisher และ service server ตาม Type ของ msg และ srv ที่กำหนดไว้ โดยชื่อ Topic และ service จะใช้ตัวแปร noise_type ทีดึงค่าตัวแปรมาจาก namespace จากนั้นประกาศ timer ที่ใช้ในการ publish msg โดยใช้คาบที่ได้มาจาก rate

        # Publishers
        self.noise_pub = self.create_publisher(Float64, f'{self.noise_type}/noise', 10)
        
        # Services Server
        self.noise_server = self.create_service(SetNoise, f'{self.noise_type}/set_noise', self.set_noise_callback)
        
        # Timer
        self.timer = self.create_timer(self.noise_rate, callback=self.timer_callback)

5.ประกาศตัวแปรไว้สำหรับเก็บค่า mean variance และ data ของ noise

        # additional attributes
        # variables
        self.noise_data = 0.0
        self.mean = 0.0
        self.variance = 1.0

6.ประกาศ function ที่ใช้ในการ handle กับ service /set_noise โดยจะทำการนำค่าที่ได้จาก srv มาเก็บไว้ในตัวแปร mean และ variance โดยจะต้องมีการ return response เสมอในการใช้งาน service

    def set_noise_callback(self,request:SetNoise.Request,response:SetNoise.Response):
        # SetNoise.srv
        # request = float64 mean , float64 variance
        # response = None
        self.mean = request.mean.data
        self.variance = request.variance.data
        return response # return response to the client